# to do:
- DONE: Paginate
- Figure out what's up with Mutual CSV formatting issues
- Remove extra lines in the CSV
- Why do you have to run the print-to-CSV part twice before the CSV appears to update? May be issue with Excel.
- Check to see if assumptions about notes aren't correct

In [ ]:
# Import some stuff you'll use
import requests
import json
import csv

# Change these: Add your credentials

In [ ]:
USER = 'Rachel'
PASS = 'Gelman#2130'
HOST = 'http://archivesspace.library.gwu.edu:8089' #don't end with a slash
def aspace_auth(host, username, password):
    auth = requests.post(HOST + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)
print(headers)

# Search for any aspace records of any type with a given query term in any notes field

In [ ]:
# CHANGE YOUR SEARCH TERM AND PAGE SIZES HERE!
q = 'holdings'
LAST_PAGE = 6
PAGE_SIZE = 10



In [ ]:
# Set things up so the filename is based on your search term
filename = "search_for_" + q + ".csv"

# Create and open the file 
f=csv.writer(open(filename, 'w'),delimiter=',',quoting=csv.QUOTE_MINIMAL)

#Create the first row of column headers
f.writerow(['uri','title','identifier','external_id','record_type','note_with_search_term','note_type'])

In [ ]:
# LET'S TEST!

#Reset page to "1"
page = 1
page_num = str(page)

#While page is less than whatever you entered above as the value for "LAST_PAGE"
while page <= LAST_PAGE:

    # Here is the actual search we're sending to ArchivesSpace!
    # First we set up the crazy complicated URL for an advanced query of the "notes" field.
    endpoint = '/repositories/2/search?page=' + page_num + '&aq={"query":{"field":"notes","value":"' + q + '", "jsonmodel_type":"field_query","negated":false,"literal":true}}'
    # Then we send that search up to ArchivesSpace
    results = requests.get(HOST + endpoint, headers=headers)
    # Make sure our python script knows to read the results as JSON
    results=results.json()


    # The results include a bunch of facet & page information, which we want to skip. 
    # So, let's just make results equal to the part of results without the facet or page info
    results=results['results']


    # Add all the data!
    # For each record returned by the search above . . . 
    for result in results:
        # Define all the fields that you'd like put into the CSV. 
        uri = result.get('uri')
        title = result.get('title')
        identifier = result.get('identifier')
        external_id = result.get('external_id')
        record_type = result.get('types')
        aspace_json = json.loads(result.get('json'))
        # Below, we are making the assumption that each record will have the search term in only one note, 
        # and each multi-part note actually only has one subnote 
        # This returns the content of the first note that has the search term.
        notes = aspace_json['notes']
        for note in notes:
            try:
                subnotes = note.get('subnotes')
                content = subnotes[0]['content']
                note_type = note['type']
            except:
                content = note['content'][0]
                note_type = note['type']
            if q in content:
                note_with_search_term = content
        f.writerow([uri,title,identifier,external_id,record_type,note_with_search_term,note_type])
    # finally, add 1 to the page so the next time the loop
    # runs it will get the next page
    page = page + 1
    page_num = str(page)

# ASpace search queries that didn't work. 

In [ ]:
query="q:Anything"
search = requests.get(HOST + '/search?', headers=headers, data=query)
search

In [ ]:
endpoint = '/search?page=1&page_size=1&q="/repositories/2"'

results = requests.get(HOST + endpoint, headers=headers)
results